# Advent of Code Challenges - Day 9

## Import Libraries

In [1]:
import numpy as np
import pandas as pd

from pandas import Series,DataFrame

## Load Data

In [2]:
input_path = './input.txt'

with open(input_path) as f:
    lines = f.readlines()
f.close()

# apply a border around the input matrix of all '9's to allow a 3x3 matrix to convolve over all matrix points
matrix = [[9] + [int(x) for x in line.replace("\n","")] + [9] for line in lines]
matrix = [[9]*102] + matrix + [[9]*102]

## Define Functions

In [3]:
def convolve_matrix(matrix):
    """Function to convolve over a matrix to find low_points (points with a value lower than the surrounding points)"""
    low_point_mtx = []
    for y in range(1,len(matrix)-1,1):
        row = []
        for x in range(1,len(matrix)-1,1):
            nearby_array =[matrix[y-1][x]]+[matrix[y][x-1]]+[matrix[y][x+1]]+[matrix[y+1][x]]
            row.append(matrix[y][x] if min(nearby_array) > matrix[y][x] else -1)
        low_point_mtx.append(row)
    return low_point_mtx

In [4]:
def get_surrounding_points(df,point):
    """Function to get all points surrounding a provided point in a provided dataframe with a value < 9"""
    x,y = point
    nearby_points = [(x-1,y), (x+1,y), (x,y-1), (x,y+1)]
    nearby_basin_points = [x for x in nearby_points if df.iloc[x[0],x[1]] < 9]
    return nearby_basin_points

In [5]:
def explore_basin(df,point):
    """Function with looping logic to search around a low point for surrounding basin points"""
    init_set_len = 1
    surrounding_points = get_surrounding_points(df,point)
    basin_points = [point] + surrounding_points
    set_len = len(basin_points)
    while set_len > init_set_len:
        init_set_len = set_len
        for point in basin_points:
            surrounding_points = get_surrounding_points(df,point)
            basin_points += surrounding_points
            basin_points = list(set(basin_points))
            set_len = len(basin_points)
    return basin_points

In [6]:
def get_basin_size(df,low_point):
    """Function to return size of a basin surrounding a low point."""
    basin_points = explore_basin(df,low_point)
    basin_size = len(basin_points)
    return basin_size

## Puzzle 1 - Smokin's bad, m-kay!
In this problem, we are trying to detect low points in a numerical representation of a topographical map of the lava tubes we're traveling through (WHAT?!)  Smoke pools in the low points, so we want to avoid them.  A 'low point' is defined as a point that is lower than its surrounding points (up, down, left, right - no, that's not a Nintendo cheat code).  To address this problem, we are going to convolve over a matrix to produce an array of values for each point on the matrix, excluding the border. The array will include the point above, below, left and right of the center point (4 points total). If the minimum of this array is greater than the center point, then that center point is a 'low point'. For each point, we will return either a -1 if that point is NOT a low point, or the point value, if it IS a low point. This will be used to create a DataFrame.

We will then add 1 to all values in the resulting DataFrame and sum the values to get our answer.

In [7]:
low_point_mtx = convolve_matrix(matrix)
low_point_df = DataFrame(low_point_mtx)
answer = (low_point_df + 1).sum().sum()
print(f"Answer: {answer}")

Answer: 478


## Puzzle 2 - Feelin low
We now need to evaluate how large the basins are. The basins are the areas surrounding the low points that have a height less than 9.  The strategey here is to start with the low point and access all of the surrounding points to see if their height is less than 9. For each surrounding point whose height is less than 9, we add that point to the set of points that belong to the basin. We then loop through this process for the larger pool of points until the set size stops increasing.

In [8]:
df = DataFrame(matrix)
low_points = []
for x in range(0,100,1):
    for y in range(0,100,1):
        if low_point_df.iloc[x,y] > -1:
            low_points.append((x+1,y+1))
low_pt_df = DataFrame(Series(low_points))
low_pt_df.columns = ['low_point']
low_pt_df['basin_size'] = [get_basin_size(df,x) for x in low_pt_df['low_point']]
low_pt_df.sort_values('basin_size', ascending=False, inplace=True)
low_pt_df.reset_index(drop=True, inplace=True)
three_basin_sizes = low_pt_df.iloc[0:3,1].values
answer = three_basin_sizes[0] * three_basin_sizes[1] * three_basin_sizes[2]
print(f"Answer: {answer}")

Answer: 1327014
